In [77]:
d = []
v = []
repeat=[]
def unique_file(input_filename):
    input_file = open(input_filename, 'r')
    file_contents = input_file.readlines()
    for line in file_contents:
        line1 = line
        if len(line1) < 2:
            continue
        if line1[-1] == ';':
            line1=line1[:-1]
        if line1[-2] == ';':
            line1=line1[:-2]
        if ':' not in line1:
            line4=line1.split('|')
            for i in range(len(line4)):
                line4[i]=line4[i].strip()
                line5 = line4[i].split(" ")
                for i in line5:
                    if i not in v:
                        v.append(i)
            continue
        line2,line3 = line1.split(':')
        line2=line2.strip()
        if line2 not in d:
            d.append(line2)
        else:
            repeat.append(line2)
        line4=line3.split('|')
        for i in range(len(line4)):
            line4[i]=line4[i].strip()
            line5 = line4[i].split(" ")
            for i in line5:
                if i not in v:
                    v.append(i)

unique_file('input.txt')
res = []
# with open('output.txt', 'w') as out:
for i in v:
    if i not in d:
        # out.write(i + '\n')
        res.append(i)
res.pop(0)
print(res)

['Identifier', 'Dot', 'Lt', 'Gt', 'Comma', 'Atr', 'Override', 'Public', 'Protected', 'Private', 'Static', 'Abstract', 'Final', 'Native', 'Synchronized', 'Transient', 'Volatile', 'Default', 'Void', 'Lsb', 'Rsb', 'Lb', 'Rb', 'Bboolean', 'Bbyte', 'Short', 'Int', 'Long', 'Char', 'Float', 'Double', 'Package', 'Semicol', 'Import', 'Mult', 'Class', 'Extends', 'Implements', 'Lcb', 'Rcb', 'Eq', 'throws', 'This', 'Super', 'Interface', 'Col', 'If', 'Else', 'Switch', 'Case', 'While', 'Do', 'For', 'Var', 'Break', 'Continue', 'Return', 'Throw', 'Try', 'Catch', 'finally', 'Literal', 'New', 'Inc', 'Dec', 'Plus', 'Minus', 'Tilde', 'Not', 'Div', 'Mod', 'Shifter', 'Relop', 'Instanceof', 'Eqnq', 'And', 'Xor', 'Or', 'Bool_and', 'Bool_or', 'Qm', 'Eqq']


In [ ]:
#To process input
def make_productions(input_filename):
    with open(input_filename, 'r') as input_file:
        with open('output.txt', 'w') as out:
            file_contents = input_file.readlines()
            # open = 0
            for line in file_contents:
                # out.write(line)
                if ':' in line:
                    line1, line2 = line.split(':')
                    out.write(line1.strip() + ': \n')
                    if len(line2.strip()) == 0:
                        continue
                elif line[0] == '|':
                    line2 = line[1:]
                else:
                    line2 = line
                for body in line2.strip().split('|'):
                    out.write('| ' + body.strip() + '\n')

make_productions('input.txt')

In [75]:
#remove '|' from 1st production
def make_productions(input_filename):
    with open(input_filename, 'r') as input_file:
        with open('output.txt', 'w') as out:
            file_contents = input_file.readlines()
            remove_first=False
            for line in file_contents:
                if line[0] != '|':
                    remove_first = True
                elif remove_first:
                    line = line[2:]
                    remove_first = False
                out.write(line)


make_productions('input.txt')

In [63]:
#Script for producing code for parse tree
def make_ast_rules(input_filename):
    with open(input_filename, 'r') as input_file:
        with open('output.txt', 'w') as out:
            file_contents = input_file.readlines()
            for line in file_contents:
                if ':' in line:
                    name = line[:-3]
                    out.write(line.strip() + '\n')
                else:
                    if line[0] == '|':
                        line1 = line[1:].strip().split(' ')
                    else:
                        line1 = line.strip().split(' ')
                    out.write(line.strip() + ' { $$ = nodecount++; labels[$$] = "' + name + '";\n')
                    i = 1
                    for var in line1:
                        j = str(i)
                        if var in res:
                            out.write('($'+j+').num = nodecount++; labels[($'+j+').num] = ($'+j+').str; from.pb($$); to.pb(($'+j+').num);\n')
                        else:
                            out.write('from.pb($$); to.pb($'+j+');\n')
                        i += 1
                    out.write('}\n')

make_ast_rules('input.txt')

In [78]:
#Script for producing code for ast (by removing parent edge in the case of single transition)
def make_ast_rules(input_filename):
    with open(input_filename, 'r') as input_file:
        with open('output.txt', 'w') as out:
            file_contents = input_file.readlines()
            for line in file_contents:
                if ':' in line:
                    name = line[:-3]
                    out.write(line.strip() + '\n')
                else:
                    if line[0] == '|':
                        line1 = line[1:].strip().split(' ')
                    else:
                        line1 = line.strip().split(' ')

                    if len(line1) == 1:
                        if line1[0] in res:
                            out.write(line.strip() + ' { ($1).num = nodecount++; labels[($1).num] = ($1).str; $$ = ($1).num; }\n')
                        elif line1[0] == '':
                            out.write(line.strip() + ' { $$ = nodecount++; labels[$$] = "' + name + '"; }\n')
                        else:
                            out.write(line.strip() + ' { $$=$1; }\n')
                    else:
                        out.write(line.strip() + ' { $$ = nodecount++; labels[$$] = "' + name + '";\n')
                        i = 1
                        for var in line1:
                            j = str(i)
                            if var in res:
                                out.write('($'+j+').num = nodecount++; labels[($'+j+').num] = ($'+j+').str; from.pb($$); to.pb(($'+j+').num);\n')
                            else:
                                out.write('from.pb($$); to.pb($'+j+');\n')
                            i += 1
                        out.write('}\n')

make_ast_rules('input.txt')

In [84]:
#Script for producing code for ast2 taking care for expressions
operator_list = ['Lt', 'Gt', 'Plus', 'Minus', 'Mult', 'Div', 'Mod', 'Shifter', 'Eqnq', 'And', 'Xor', 'Or', 'Bool_and', 'Eq', 'Eqq', 'Bool_or', 'Relop']
def make_ast_rules(input_filename):
    with open(input_filename, 'r') as input_file:
        with open('output.txt', 'w') as out:
            file_contents = input_file.readlines()
            for line in file_contents:
                if ':' in line:
                    name = line[:-3]
                    out.write(line.strip() + '\n')
                else:
                    if line[0] == '|':
                        line1 = line[1:].strip().split(' ')
                    else:
                        line1 = line.strip().split(' ')

                    if len(line1) == 1:
                        if line1[0] in res:
                            out.write(line.strip() + ' { ($1).num = nodecount++; labels[($1).num] = ($1).str; $$ = ($1).num; }\n')
                        elif line1[0] == '':
                            out.write(line.strip() + ' { $$ = nodecount++; labels[$$] = "' + name + '"; }\n')
                        else:
                            out.write(line.strip() + ' { $$=$1; }\n')
                    elif len(line1) == 3 and line1[1].strip() in operator_list:
                        print(line1[1].strip())
                        out.write(line.strip() + ' { ($2).num = nodecount++; labels[($2).num] = ($2).str; $$ = ($2).num;\n')
                        if line1[0] in res:
                            out.write('($1).num = nodecount++; labels[($1).num] = ($1).str; from.pb($$); to.pb(($1).num);\n')
                        else:
                            out.write('from.pb($$); to.pb($1);\n')
                        if line1[2] in res:
                            out.write('($3).num = nodecount++; labels[($3).num] = ($3).str; from.pb($$); to.pb(($3).num);\n')
                        else:
                            out.write('from.pb($$); to.pb($3);\n')
                        out.write('}\n')
                    else:
                        out.write(line.strip() + ' { $$ = nodecount++; labels[$$] = "' + name + '";\n')
                        i = 1
                        for var in line1:
                            j = str(i)
                            if var in res:
                                out.write('($'+j+').num = nodecount++; labels[($'+j+').num] = ($'+j+').str; from.pb($$); to.pb(($'+j+').num);\n')
                            else:
                                out.write('from.pb($$); to.pb($'+j+');\n')
                            i += 1
                        out.write('}\n')

make_ast_rules('input.txt')

Eq
Mult
Div
Mod
Plus
Minus
Shifter
Lt
Gt
Relop
Eqnq
And
Xor
Or
Bool_and
Bool_or


In [80]:
operator_list = ['LT', 'GT', 'PLUS', 'MINUS', 'MULT', 'DIV', 'MOD', 'SHIFTER', 'EQNQ', 'AND', 'XOR', 'OR', 'BOOL_AND', 'EQ', 'EQQ', 'BOOL_OR', 'RELOP']
len(operator_list)

17